In [1]:
using JuMP, GLPK

In [2]:
# Declare the predefined parameters
cnorm=32.0 # production cost of a bike in normal hours
cover=40.0 # the production cost of a bike in overtime hours
cstock=5.0 # the monthly storage cost per bike
ISTOCK=2.0 # the initial stock level
CAP=30 # production capacity per month

demand=[30.0, 15.0, 15.0, 25.0, 33.0, 40.0, 45.0, 45.0, 26.0, 14.0, 25.0, 30.0]; # the forecasted demand
n=length(demand);

In [3]:
# Linear programming optimization
model=Model(with_optimizer(GLPK.Optimizer));

In [4]:
# decision varaibles
@variable(model,pnorm[1:n], lower_bound=0, upper_bound=CAP); # number of bikes to be produced in normal working hours during month t.
@variable(model,pover[1:n], lower_bound=0, upper_bound=0.5*CAP); # number of bikes to be produced in overtime hours during month t.
@variable(model,store[0:n]); # number of bikes held in stock at the end of the month t.
fix(store[0], ISTOCK; force = true)

In [5]:
# objective function
@objective(model,Min,sum(cnorm*pnorm[i]+cover*pover[i]+cstock*store[i] for i in 1:n))     

32 pnorm[1] + 40 pover[1] + 5 store[1] + 32 pnorm[2] + 40 pover[2] + 5 store[2] + 32 pnorm[3] + 40 pover[3] + 5 store[3] + 32 pnorm[4] + 40 pover[4] + 5 store[4] + 32 pnorm[5] + 40 pover[5] + 5 store[5] + 32 pnorm[6] + 40 pover[6] + 5 store[6] + 32 pnorm[7] + 40 pover[7] + 5 store[7] + 32 pnorm[8] + 40 pover[8] + 5 store[8] + 32 pnorm[9] + 40 pover[9] + 5 store[9] + 32 pnorm[10] + 40 pover[10] + 5 store[10] + 32 pnorm[11] + 40 pover[11] + 5 store[11] + 32 pnorm[12] + 40 pover[12] + 5 store[12]

In [6]:
# constraints
@constraint(model,[i=1:n],pnorm[i]+pover[i]+store[i-1]==demand[i]+store[i]);
@constraint(model,[i=1:n],store[i]>=0);

In [7]:
# solve optimization problem
JuMP.optimize!(model)

In [9]:
# print optimal values
JuMP.value.(pnorm)

12-element Array{Float64,1}:
 28.0
 15.0
 15.0
 28.0
 30.0
 30.0
 30.0
 30.0
 26.0
 14.0
 25.0
 30.0

In [10]:
JuMP.value.(pover)

12-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
 10.0
 15.0
 15.0
  0.0
  0.0
  0.0
  0.0

In [11]:
JuMP.value.(store)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:12
And data, a 13-element Array{Float64,1}:
 2.0
 0.0
 0.0
 0.0
 3.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
#print objective function value
print(JuMP.objective_value(model))

11247.0